## This notebook compares the embeddings generated by OpenAIEmbeddings.

It shows that OpenAIEmbeddings embeddings can differ slightly (typically at 4 the decimal place).

### Results from OpenAIEmbeddings:
encodings are NOT identical on each run.

### Repeating with sentence-transformers/all-MiniLM-L6-v2:
encodings ARE identical on each run.

Tests verify simple numerical comparisons.

### Advanced Comparison
A more advanced euclidean and cosine comparison is also included.

## NOTES: Tests run on local Jupiter Notebook| Anaconda setup for the course.

In [ ]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [ ]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [ ]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [ ]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
len(chunks)

In [ ]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

In [ ]:
# Use a where filter to specify the metadata condition
# Get the 3 company vectors (corresponds to our 3 yellow dots)

def get_company_vectors(collection):
    company_vectors = collection.get(
        where={"doc_type": "company"},  # Filter for documents where source = "XXXX"
        limit=10,
        include=["embeddings", "metadatas", "documents"]
    )
    print(f"Found {len(company_vectors)} company vectors")
    return company_vectors


In [ ]:

def print_vectors_summary(vectors):
    for i in range(len(vectors["documents"])):
        print(f"\n--- Chunk {i+1} ---")
        
        # Print document content (first 100 chars)
        print(f"Content: {vectors['documents'][i][:100]}...")
        
        # Print metadata
        print(f"Metadata: {vectors['metadatas'][i]}")
        
        # Print embedding info (not the full vector as it would be too long)
        embedding = vectors["embeddings"][i]
        print(f"Embedding: Vector of length {len(embedding)}, first 5 values: {embedding[:5]}")


def get_dimensions_for_vectors(vectors):
    dimensions = []

    for i in range(len(vectors["documents"])):
        embedding = vectors["embeddings"][i]
        dimensions.append(embedding)

    return dimensions


In [ ]:
# Quick check if any single value is different
def quick_diff_check(emb1, emb2):
    result = "Embeddings are identical"
    print("\n\nComparing two embeddings:\n\n")
    print(emb1)
    print(emb2)
    for i, (v1, v2) in enumerate(zip(emb1, emb2)):
        if v1 != v2:
            result = f"Different at dimension {i}: {v1} vs {v2}"
            break
    print(result)
    return result

#quick_diff_check(dimensions[0], dimensions[1])

In [ ]:

embeddings = OpenAIEmbeddings()

def create_vectorstores(embeddings):

    if os.path.exists("vectorstore1"):
        Chroma(persist_directory="vectorstore1", embedding_function=embeddings).delete_collection()
    if os.path.exists("vectorstore2"):
        Chroma(persist_directory="vectorstore2", embedding_function=embeddings).delete_collection()
    
    
    # Create vectorstore 1
    vectorstore1 = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="vectorstore1")
    print(f"Vectorstore 1 created with {vectorstore1._collection.count()} documents")
    
    # Create vectorstore 2
    vectorstore2 = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="vectorstore2")
    print(f"Vectorstore 2 created with {vectorstore2._collection.count()} documents")

    return vectorstore1, vectorstore2

vectorstore1, vectorstore2 = create_vectorstores(embeddings)



In [ ]:
## Uncomment this and rerun cells below, 
## to see that HuggingFaceEmbeddings is idential

#from langchain.embeddings import HuggingFaceEmbeddings
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#vectorstore1, vectorstore2 = create_vectorstores(embeddings)


In [ ]:
# Get the 3 company doc_type vectors
collection1 = vectorstore1._collection
collection2 = vectorstore2._collection

company_vectors1=get_company_vectors(collection1)
company_vectors2=get_company_vectors(collection2)


In [ ]:
# Lets print out summary info just to see we have the same chunks.

def print_summary_info (vectors):
    print("VECTORS SUMMARY\n")
    print_vectors_summary(vectors)


print("\n\n\n========= VECTORS 1 =========\n\n")
print_summary_info(company_vectors1)

print("\n\n\n========= VECTORS 2 =========\n\n")
print_summary_info(company_vectors2)




In [ ]:
dimensions1 = get_dimensions_for_vectors(company_vectors1)
dimensions2 = get_dimensions_for_vectors(company_vectors2)

result1 = quick_diff_check(dimensions1[0], dimensions2[0])   
result2 = quick_diff_check(dimensions1[1], dimensions2[1])    
result3 = quick_diff_check(dimensions1[2], dimensions2[2])    

print("\n\nSUMMARY RESULTS:")
print("================\n\n")
print(result1)    
print(result2)
print(result3)


In [ ]:
## ADVANCED COMPARISONS:
# More advanced comparisons (from Claude 3.7 Sonnet):


## !IMPORTANT *** Uncomment final line to execute ***


import numpy as np
from scipy.spatial.distance import cosine

# Method 1: Euclidean distance (L2 norm)
def compare_embeddings_euclidean(emb1, emb2):
    emb1_array = np.array(emb1)
    emb2_array = np.array(emb2)
    distance = np.linalg.norm(emb1_array - emb2_array)
    return {
        "different": distance > 0,
        "distance": distance,
        "similarity": 1/(1+distance)  # Converts distance to similarity score
    }

# Method 2: Cosine similarity (common for embeddings)
def compare_embeddings_cosine(emb1, emb2):
    emb1_array = np.array(emb1)
    emb2_array = np.array(emb2)
    similarity = 1 - cosine(emb1_array, emb2_array)  # Cosine returns distance, so subtract from 1
    return {
        "different": similarity < 0.9999,  # Almost identical if > 0.9999
        "similarity": similarity
    }

# Method 3: Simple exact equality check
def are_embeddings_identical(emb1, emb2):
    return np.array_equal(np.array(emb1), np.array(emb2))


def run_advanced_comparisons():
    for i in range(0, 3):
        print(f"\n\nComparing vector dimensions for dimension[{i}]....\n")
        print("Exactly identical?    ---> ", are_embeddings_identical(dimensions1[i], dimensions2[i]))
        print("Cosine comparison:    ---> ", compare_embeddings_cosine(dimensions1[i], dimensions2[i]))
        print("Euclidean comparison: ---> ", compare_embeddings_euclidean(dimensions1[i], dimensions2[i]))


#run_advanced_comparisons()